In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import psycopg2
import time
import math
from datetime import date, timedelta, datetime

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.model_selection as model_sel

# Import data

In [2]:
with open ('/Users/patriciosanchez/Documents/Python/Connections/dwh.txt', 'r') as file:
    lines=file.readlines()
    dwh_host=lines[0].strip('\n')
    dwh_user=lines[1].strip('\n')
    dwh_port=lines[2].strip('\n')
    dwh_database=lines[3].strip('\n')
    dwh_password=lines[4].strip('\n')
    file.close()

In [3]:
con = psycopg2.connect(dbname= dwh_database,
                    host=dwh_host,
                    port= dwh_port,
                    user= dwh_user,
                    password= dwh_password)


# Define timeframe
start_date = date(2021, 7, 2)
end_date = date(2021, 8, 9)
delta = timedelta(days=1)
data = pd.DataFrame()


# Loop all days
while start_date < end_date:
    print (start_date.strftime("%Y-%m-%d"))
    next_date = start_date+delta
    
    qry = '''select     o.id,
           o.customer_id,
           o.creation_time,
           date_trunc('day', o.creation_time) as "day_creation_time",
           o.country_code,
           o.city_code,
           p.payment_status,
           o.final_status,
           p.forgiven,
           p.bundled_payment_parent_id,
           o.gtv/o.exchange_rate_currency as order_gtv,
           cast(json_extract_path_text(features, 'customer', 'debtTotalInEur') as float8) as debt,
           (order_gtv - debt) as gtv_minus_debt,
           nullif(json_extract_path_text(features, 'ml', 'ml_uch', 'score'),' ')::int as score,
           json_extract_path_text(features, 'customer', 'finishedOrdersCount') as checkout_gtv
            from risk_feature_snapshots rfs
            join checkout_order co on json_extract_path_text(features, 'order', 'checkoutId') = cast(co.checkout_id as text)
            join orders o on o.id = co.order_id
            join payments p on p.order_id = o.id
            join users u on u.id = o.customer_id
            where 1=1
            and rfs.checkpoint = 'checkout'
            --and o.country_code in ('CI')
            and o.ordering_payment_method = 'CASH'
            and o.final_status in ('DeliveredStatus', 'CanceledStatus')
            and score is not null
            and u.staff = False
            and o.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '\' and o.creation_time < \'' + next_date.strftime("%Y-%m-%d") + '\''

    df = pd.read_sql_query(qry, con)
    data = data.append(df)
    start_date = next_date
    
con.close()

2021-07-02
2021-07-03
2021-07-04
2021-07-05
2021-07-06
2021-07-07
2021-07-08
2021-07-09
2021-07-10
2021-07-11
2021-07-12
2021-07-13
2021-07-14
2021-07-15
2021-07-16
2021-07-17
2021-07-18
2021-07-19
2021-07-20
2021-07-21
2021-07-22
2021-07-23
2021-07-24
2021-07-25
2021-07-26
2021-07-27
2021-07-28
2021-07-29
2021-07-30
2021-07-31
2021-08-01
2021-08-02
2021-08-03
2021-08-04
2021-08-05
2021-08-06
2021-08-07
2021-08-08


In [4]:
con = psycopg2.connect(dbname= dwh_database,
                    host=dwh_host,
                    port= dwh_port,
                    user= dwh_user,
                    password= dwh_password)

start_date = date(2021, 7, 2)
next_date = date(2021, 8, 9)

qry2 = '''with blocked as (
    select json_extract_path_text(features, 'order', 'countryCode') as country_code,
           count(distinct (json_extract_path_text(features, 'customer', 'id'))) as blocked_customers
    from risk_feature_snapshots rfs
             join checkout_order co
                  on json_extract_path_text(features, 'order', 'checkoutId') = cast(co.checkout_id as text)
             join risk_decisions rd on rd.decision_id = rfs.decision_id
    where 1 = 1
      and  rfs.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '''\'
      and  rfs.creation_time < \'''' + next_date.strftime("%Y-%m-%d") + '''\'
      and rd.rule in ('ml_cash_unpaid', 'block_cash_multi_order')
    and rd.shadowing is False
group by 1
),
non_blocked as (
         select country_code,
                count(distinct (customer_id)) as customers
         from (select o.country_code as country_code,
                      o.customer_id as customer_id
               from orders o
               where 1 = 1
      and  o.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '''\'
      and  o.creation_time < \'''' + next_date.strftime("%Y-%m-%d") + '''\'
            union
            select json_extract_path_text(features, 'order', 'countryCode') as country_code,
                    nullif(json_extract_path_text(features, 'customer', 'id'), '')::int as customer_id
                 from risk_feature_snapshots rfs
                join checkout_order co
                  on json_extract_path_text(features, 'order', 'checkoutId') = cast(co.checkout_id as text)
                join risk_decisions rd on rd.decision_id = rfs.decision_id
            where 1 = 1
      and  rfs.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '''\'
      and  rfs.creation_time < \'''' + next_date.strftime("%Y-%m-%d") + '''\'
              and rd.rule = 'ml_cash_unpaid'
               and rd.shadowing is False
                )
         group by 1
     ),
non_blocked_cash as (
         select country_code,
                count(distinct (customer_id)) as customers_cash
         from (select o.country_code as country_code,
                      o.customer_id as customer_id
               from orders o
               where 1 = 1
               and o.ordering_payment_method = 'CASH'
      and  o.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '''\'
      and  o.creation_time < \'''' + next_date.strftime("%Y-%m-%d") + '''\'
            union
            select json_extract_path_text(features, 'order', 'countryCode') as country_code,
                    nullif(json_extract_path_text(features, 'customer', 'id'), '')::int as customer_id
                 from risk_feature_snapshots rfs
                join checkout_order co
                  on json_extract_path_text(features, 'order', 'checkoutId') = cast(co.checkout_id as text)
                join risk_decisions rd on rd.decision_id = rfs.decision_id
            where 1 = 1
      and  rfs.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '''\'
      and  rfs.creation_time < \'''' + next_date.strftime("%Y-%m-%d") + '''\'
              and rd.rule = 'ml_cash_unpaid'
               and rd.shadowing is False
                )
         group by 1
     )
select blocked.country_code,
       blocked.blocked_customers,
       non_blocked.customers,
       non_blocked_cash.customers_cash,
       (100.0*blocked_customers)/(1.0*non_blocked.customers) as block_rate,
       (100.0*blocked_customers)/(1.0*non_blocked_cash.customers_cash) as block_rate_cash
from blocked
join non_blocked on non_blocked.country_code = blocked.country_code
join non_blocked_cash on non_blocked_cash.country_code = blocked.country_code'''    

data_general = pd.read_sql_query(qry2, con)

con.close()

In [5]:
def main_loop(c, th_grid, th_c, df_country, th_limit):
    
        # Loop through threshold values. Compute true positive rate (tp) and false positive rate (fp)
        for i in th_grid:
                    df = df_country[(df_country['score'] >= i.get('score'))]
                    tp = df[df['payment_status']=='NOT_PAID']['id'].count()
                    fp = df[df['payment_status']=='PAID']['id'].count()
                    tp_gtv = df[df['payment_status']=='NOT_PAID']['order_gtv'].sum() - df[df['payment_status']=='NOT_PAID']['debt'].sum()
                    fp_gtv = df[df['payment_status']=='PAID']['order_gtv'].sum() - df[df['payment_status']=='PAID']['debt'].sum()
                    tp_customers = df[df['payment_status']=='NOT_PAID']['customer_id'].nunique()
                    fp_customers = df[df['payment_status']=='PAID']['customer_id'].nunique()

                    if tp !=0 and fp!= 0:
                        precision = 100*tp/(tp+fp)
                        precision_gtv = 100*tp_gtv/(tp_gtv+fp_gtv)
                        precision_customers = 100*tp_customers/(tp_customers+fp_customers)
                        recall = 100*tp/(df_country[df_country['payment_status']=='NOT_PAID']['id'].count())
                        recall_gtv = 100*tp_gtv/(df_country[df_country['payment_status']=='NOT_PAID']['order_gtv'].sum()-df_country[df_country['payment_status']=='NOT_PAID']['debt'].sum())
                        
                        if precision_gtv >= th_limit:
                            th_c = th_c.append({'country_code': str(c),
                                            'thresholds':str(i), 
                                            'tp': float(tp),
                                            'fp': float(fp),
                                            'tp_gtv': float(tp_gtv),
                                            'fp_gtv': float(fp_gtv),
                                            'tp_customers': float(tp_customers),
                                            'fp_customers': float(fp_customers),                                                
                                            'precision': float(precision),
                                            'precision_gtv': float(precision_gtv),
                                            'precision_customers': float(precision_gtv),
                                            'recall': float(recall),
                                            'recall_gtv': float(recall_gtv)
                                            }, ignore_index=True) 
        return th_c


def optimize_th(df_input):
    
    th_grid = model_sel.ParameterGrid({"score": list(range(0, 100))})
    
    th = pd.DataFrame({'country_code': pd.Series([], dtype='object'),
                           'thresholds': pd.Series([], dtype='object'),
                           'tp': pd.Series([], dtype='float'),
                           'fp': pd.Series([], dtype='float'),
                           'tp_gtv': pd.Series([], dtype='float'),
                           'fp_gtv': pd.Series([], dtype='float'),
                           'tp_customers': pd.Series([], dtype='float'),
                           'fp_customers': pd.Series([], dtype='float'),                       
                           'precision': pd.Series([], dtype='float'),
                           'precision_gtv': pd.Series([], dtype='float'),
                           'precision_customers': pd.Series([], dtype='float'),
                           'recall': pd.Series([], dtype='float'), 
                           'recall_gtv': pd.Series([], dtype='float')
                           })
    
    for c in df_input['country_code'].unique():
        
        print(c)
        df_country = df_input[df_input['country_code']==c]
        
        th_c = pd.DataFrame({'country_code': pd.Series([], dtype='object'),
                           'thresholds': pd.Series([], dtype='object'),
                           'tp': pd.Series([], dtype='float'),
                           'fp': pd.Series([], dtype='float'),
                           'tp_gtv': pd.Series([], dtype='float'),
                           'fp_gtv': pd.Series([], dtype='float'),
                           'tp_customers': pd.Series([], dtype='float'),
                           'fp_customers': pd.Series([], dtype='float'),    
                           'precision': pd.Series([], dtype='float'),
                           'precision_gtv': pd.Series([], dtype='float'),
                           'precision_customers': pd.Series([], dtype='float'),
                           'recall': pd.Series([], dtype='float'), 
                           'recall_gtv': pd.Series([], dtype='float') 
                           })

        th_c = main_loop(c, th_grid, th_c, df_country, 0.1)
            
        th = th.append(th_c.sort_values(by=['thresholds'], ascending=False))
    
    return th

In [6]:
th_markets = optimize_th(data)

results = th_markets.merge(data_general, how='inner', left_on='country_code', right_on='country_code')
results['BLOCK_RATE_ALL'] = (results['tp_customers']+results['fp_customers']+results['blocked_customers'])/(results['customers']+results['blocked_customers'])
results['BLOCK_RATE_CASH'] = (results['tp_customers']+results['fp_customers']+results['blocked_customers'])/(results['customers_cash']+results['blocked_customers'])

RO
KZ
UG
GE
UA
CI
MA
BG
IT
HR
PT
KE
BA
PL
ES
RS
KG
GH
ME
MD
SI
NG


In [7]:
results

,country_code,thresholds,tp,fp,tp_gtv,fp_gtv,tp_customers,fp_customers,precision,precision_gtv,precision_customers,recall,recall_gtv,blocked_customers,customers,customers_cash,block_rate,block_rate_cash,BLOCK_RATE_ALL,BLOCK_RATE_CASH
0,RO,{'score': 9},1130.0,34300.0,15975.790597,454805.202319,1107.0,29820.0,3.189388,3.393466,3.393466,24.950320,29.213496,6216,480256,288732,1.294310,2.152861,0.076352,0.125931
1,RO,{'score': 93},18.0,41.0,176.812578,276.851651,17.0,40.0,30.508475,38.974327,38.974327,0.397439,0.323321,6216,480256,288732,1.294310,2.152861,0.012895,0.021268
2,RO,{'score': 92},30.0,85.0,266.319594,744.129553,29.0,84.0,26.086957,26.356556,26.356556,0.662398,0.486995,6216,480256,288732,1.294310,2.152861,0.013010,0.021458
3,RO,{'score': 91},41.0,131.0,600.636673,1315.264013,40.0,128.0,23.837209,31.350094,31.350094,0.905277,1.098330,6216,480256,288732,1.294310,2.152861,0.013123,0.021644
4,RO,{'score': 90},47.0,183.0,638.010496,1959.048816,46.0,177.0,20.434783,24.566651,24.566651,1.037757,1.166673,6216,480256,288732,1.294310,2.152861,0.013236,0.021831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,SI,{'score': 13},20.0,895.0,80.980000,6089.930000,20.0,696.0,2.185792,1.312286,1.312286,40.000000,21.150230,14,8853,4608,0.158138,0.303819,0.082328,0.157940
1921,SI,{'score': 12},20.0,932.0,80.980000,6367.560000,20.0,729.0,2.100840,1.255788,1.255788,40.000000,21.150230,14,8853,4608,0.158138,0.303819,0.086049,0.165080
1922,SI,{'score': 11},20.0,962.0,80.980000,6603.360000,20.0,753.0,2.036660,1.211488,1.211488,40.000000,21.150230,14,8853,4608,0.158138,0.303819,0.088756,0.170273
1923,SI,{'score': 10},20.0,1009.0,80.980000,6997.010000,20.0,794.0,1.943635,1.144110,1.144110,40.000000,21.150230,14,8853,4608,0.158138,0.303819,0.093380,0.179143


In [8]:
results.to_csv('ml_thresholds_20210802_20210808.csv', index = False)

# Block in specific country

con = psycopg2.connect(dbname= dwh_database,
                    host=dwh_host,
                    port= dwh_port,
                    user= dwh_user,
                    password= dwh_password)


# Define timeframe
start_date = date(2021, 7, 19)
end_date = date(2021, 8, 9)
delta = timedelta(days=1)
data_blocked = pd.DataFrame()

# Loop all days
while start_date < end_date:
    print (start_date.strftime("%Y-%m-%d"))
    next_date = start_date+delta
    
    qry = '''select json_extract_path_text(features, 'order', 'customerId') as customer_id,
               json_extract_path_text(features, 'order', 'countryCode') as country_code,
               nullif(json_extract_path_text(features, 'ml', 'ml_uch', 'score'),' ')::int as score
                from risk_feature_snapshots rfs
                join checkout_order co on json_extract_path_text(features, 'order', 'checkoutId') = cast(co.checkout_id as text)
                where 1=1
                and rfs.checkpoint = 'checkout'
                --and json_extract_path_text(features, 'order', 'countryCode') in ('CI')
                and json_extract_path_text(features, 'payment_method', 'type') = 'CASH'
                and score is not null
                and rfs.creation_time >= \'''' + start_date.strftime("%Y-%m-%d") + '\' and rfs.creation_time < \'' + next_date.strftime("%Y-%m-%d") + '\''

    df = pd.read_sql_query(qry, con)
    data_blocked = data_blocked.append(df)
    start_date = next_date
    
con.close()

data_blocks = pd.DataFrame(columns=['country_code', 'score', 'customers_blocked', 'block_rate'])
all_customers = data_blocked[(data_blocked['country_code']=='CI') & (data_blocked['score']>=0)]['customer_id'].nunique()

for c in ['CI']:
    for i in list(range(0,100)):
        blocks = data_blocked[(data_blocked['country_code']==c) & (data_blocked['score']>=i)]['customer_id'].nunique()
        data_blocks = data_blocks.append({'country_code': str(c),
                                            'score':str(i), 
                                            'customers_blocked': blocks,
                                            'block_rate': 100.0*blocks/all_customers,
                                            }, ignore_index=True) 
        
data_blocks

In [9]:
#data_blocks.to_csv('blocks_20210602_20210607.csv', index = False)